In [ ]:
# --------------------------
# Import dataset
# --------------------------
# Import dataset
import pandas as pd
import zipfile
from google.colab import files

# ===============================
# Upload zip-file from computer
uploaded = files.upload()

#=================================
#Process zip-file
# Archive name
zip_filename = "netflix_titles.zip"

# Unpack zip
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(".")
# ===============================
# Now read csv-file
# Try reading with a different encoding if UTF-8 fails
try:
    df = pd.read_csv("netflix_titles.csv", encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv("netflix_titles.csv", encoding='latin-1')

# Display first few rows to inspect
display(df.head())
print("\nDataset info:")
print(df.info())

Saving netflix_titles.zip to netflix_titles.zip


show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021          2021  TV-MA   1 Season   
3  September 24, 2021          2021  TV-MA   1 Season   
4  September 24, 2021          2021  TV-MA  2 Seasons   

                                           listed_in  \
0                                      Documentaries   
1    International TV Shows, TV Dramas, TV Mysteries   
2  Crime TV Shows, International TV Shows, TV Act...   
3                             Docuseries, Reality TV   
4  International TV Shows, Romantic TV Shows, TV ...   

                                         description  
0  As her father nears the end of his life, filmm...  
1  After crossing paths at a party, a Cape Town t...  
2  To protect his family from a powerful drug lor...  
3  Feuds, flirtations and toilet talk go down amo...  
4  In a city of coaching centers known to train I...


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
None


In [ ]:
# === Подготовка: установить библиотеки (в Colab обычно уже есть) ===
!pip install sqlalchemy --quiet

import pandas as pd
from sqlalchemy import create_engine, text
import sqlite3

SQLITE_DB_PATH = "/content/netflix.db"
TABLE_NAME = "netflix"

In [ ]:
# === 2) Создать SQLite DB через SQLAlchemy и экспортировать DataFrame ===
engine = create_engine(f"sqlite:///{SQLITE_DB_PATH}", echo=False)
df.to_sql(TABLE_NAME, con=engine, if_exists='replace', index=False)
print(f"Table '{TABLE_NAME}' written to {SQLITE_DB_PATH} (rows: {len(df)})\n")

In [ ]:
def run_sql(sql, head=10):
    """Executes SQL and outputs the result as a table."""
    with engine.connect() as conn:
        res = pd.read_sql_query(sql, conn) #returns the result as a DataFrame (pandas table).
    print(" Query completed:")
    print(sql)
    print("\n Result:")
    display(res.head(head)) #The display() function is used in Google Colab/Jupyter to beautifully display the DataFrame as a table.
    return res #Returns the entire DataFrame res so that you can continue to use the result in Python (for example, save it to CSV, filter, analyze, etc.).


In [ ]:
# === 3) Query 1: Select * ===
q1 = f"SELECT * FROM {TABLE_NAME};"
res1 = run_sql(q1, head=5)   # showing the first 5 lines

In [ ]:
# === 4) Query 2: Select Distinct (show_id) ===
q2 = f"SELECT DISTINCT show_id FROM {TABLE_NAME};"
res2 = run_sql(q2, head=20)

In [ ]:
# === 5) Query 3: Select Where (distinct titles released after 2020) ===
q3 = f"SELECT DISTINCT title, release_year FROM {TABLE_NAME} WHERE release_year > 2020 ORDER BY release_year;"
res3 = run_sql(q3, head=10)

In [ ]:
# === 6) Query 4: Select with AND, OR, NOT (examples of conditions) ===
# Example: choose distinct titles released after 2020 AND (rating='PG-13' OR rating='TV-MA') AND NOT country='South Korea'
q4 = f"""
SELECT DISTINCT title, release_year, rating, country
FROM {TABLE_NAME}
WHERE release_year > 2020
  AND (rating = 'PG-13' OR rating = 'TV-MA')
  AND NOT (country = 'South Korea')
ORDER BY release_year DESC;
"""
res4 = run_sql(q4, head=10)

In [ ]:
# === 7) Query 5: Order By (по title и release_year) ===
q5 = f"SELECT title, release_year FROM {TABLE_NAME} ORDER BY title COLLATE NOCASE ASC LIMIT 50;"
res5 = run_sql(q5, head=10)

# === 8) Query 6: Limit Values (первые 5 записей) ===
q6 = f"SELECT * FROM {TABLE_NAME} LIMIT 5;"
res6 = run_sql(q6, head=10)

In [ ]:
# === 9) Query 7: Min, Max, Count, Avg, Sum (по release_year) ===
q7 = f"""
SELECT
  MIN(release_year) AS min_year,
  MAX(release_year) AS max_year,
  COUNT(*) AS total_rows,
  ROUND(AVG(release_year), 2) AS avg_year,
  SUM(CASE WHEN release_year IS NOT NULL THEN 1 ELSE 0 END) AS sum_non_null_release_years
FROM {TABLE_NAME};
"""
res7 = run_sql(q7, head=10)

In [ ]:
# === 10) Query 8: LIKE (find countries by template) ===
# Examples: countries containing 'United' starting with 'U' ending with 'a'
q8 = f"""
SELECT DISTINCT country
FROM {TABLE_NAME}
WHERE country IS NOT NULL
  AND (
    country LIKE '%United%' OR
    (country LIKE 'U%' AND
    country LIKE '%a')
  )
ORDER BY country
LIMIT 50;
"""
res8 = run_sql(q8, head=10)

In [ ]:
# === 11) Query 9: IN (страны в конкретном списке) ===
q9 = f"""
SELECT title, country, release_year
FROM {TABLE_NAME}
WHERE country IN ('United States', 'Canada', 'United Kingdom')
LIMIT 10
"""
res9 = run_sql(q9, head=10)

In [ ]:
# === 12) Query 10: BETWEEN (release_year между 2020 и 2021) ===
q10 = f"SELECT title, release_year FROM {TABLE_NAME} WHERE release_year BETWEEN 2020 AND 2021 ORDER BY release_year;"
res10 = run_sql(q10, head=10)

In [ ]:
# === 13) Query 11: Joins (self-join) ===
# Example: find pairs of shows with the same director (by himself, different show_id)
q11 = f"""
SELECT a.show_id AS id_a, a.title AS title_a, b.show_id AS id_b, b.title AS title_b, a.director
FROM {TABLE_NAME} a
JOIN {TABLE_NAME} b
  ON a.director IS NOT NULL
  AND a.director = b.director
  AND a.show_id <> b.show_id
LIMIT 50;
"""
res11 = run_sql(q11, head=10)

In [ ]:
# === 14) Query 12: Unions ===
# Example: all headlines 2020 UNION headlines 2021 (removes duplicates)
q12 = f"""
SELECT title, release_year FROM {TABLE_NAME} WHERE release_year = 2020
UNION
SELECT title, release_year FROM {TABLE_NAME} WHERE release_year = 2021;
"""
res12 = run_sql(q12, head=10)

In [ ]:
# === 15) Query 13: CASE statements (categorization by rating) ===
q13 = f"""
SELECT title, rating,
  CASE
    WHEN rating LIKE 'TV-%' THEN 'TV Show'
    WHEN rating IN ('G','PG','PG-13') THEN 'Family'
    WHEN rating IS NULL THEN 'No rating'
    ELSE 'Adult/Other'
  END AS rating_group
FROM {TABLE_NAME}
LIMIT 50;
"""
res13 = run_sql(q13, head=10)

In [ ]:
# === 16) Query 14: Subqueries ===
# Example: select a show, release_year > average release_year
q14 = f"""
SELECT title, release_year
FROM {TABLE_NAME}
WHERE release_year > (SELECT AVG(release_year) FROM {TABLE_NAME} WHERE release_year IS NOT NULL)
ORDER BY release_year DESC
LIMIT 50;
"""
res14 = run_sql(q14, head=10)

In [ ]:
# === 17) Query 15: COALESCE (обработка NULL) ===
q15 = f"""
SELECT title, COALESCE(country, 'Unknown') AS country_or_unknown, COALESCE(director, 'No director') AS director_or_none
FROM {TABLE_NAME}
LIMIT 50;
"""
res15 = run_sql(q15, head=10)

In [ ]:
# === 18) Query 16: Convert (CAST) ===
# Example: convert release_year to text and back
q16 = f"""
SELECT title, CAST(release_year AS TEXT) AS release_year_text, LENGTH(CAST(release_year AS TEXT)) AS len_year
FROM {TABLE_NAME}
WHERE release_year IS NOT NULL
LIMIT 50;
"""
res16 = run_sql(q16, head=10)

In [ ]:
# === 19) Query 17: LAG / LEAD (window functions) ===
# Example: previous and next release by year (sorted by release_year)
q17 = f"""
SELECT title, release_year,
  LAG(title) OVER (ORDER BY release_year) AS prev_title,
  LEAD(title) OVER (ORDER BY release_year) AS next_title
FROM {TABLE_NAME}
WHERE release_year IS NOT NULL
ORDER BY release_year
LIMIT 50;
"""
res17 = run_sql(q17, head=10)

In [ ]:
# === 20) Query 18: ROW_NUMBER (window function) ===
q18 = f"""
SELECT title, release_year,
  ROW_NUMBER() OVER (ORDER BY release_year DESC) AS rn
FROM {TABLE_NAME}
WHERE release_year IS NOT NULL
LIMIT 50;
"""
res18 = run_sql(q18, head=10)

In [ ]:
# === 21) Query 19: DENSE_RANK (window function) ===
q19 = f"""
SELECT title, release_year,
  DENSE_RANK() OVER (ORDER BY release_year DESC) AS dr
FROM {TABLE_NAME}
WHERE release_year IS NOT NULL
LIMIT 50;
"""
res19 = run_sql(q19, head=10)

In [ ]:
# === 22) Query 20: WITH (CTE) ===
# Example: CTE for the filter (top directors by number of shows), then use it
q20 = f"""
WITH director_counts AS (
  SELECT director, COUNT(*) AS cnt
  FROM {TABLE_NAME}
  WHERE director IS NOT NULL
  GROUP BY director
  HAVING cnt >= 2
)
SELECT n.title, n.director, dc.cnt
FROM {TABLE_NAME} n
JOIN director_counts dc ON n.director = dc.director
ORDER BY dc.cnt DESC, n.release_year DESC
LIMIT 100;
"""
res20 = run_sql(q20, head=10)